In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 250*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240511021618
0	Validation losses: 0.5629, 51.9124, 52.4752	Best Loss: 52.4752 (0)	Accuracy: 78.22%
1	Validation losses: 0.5504, 8.7180, 9.2684	Best Loss: 9.2684 (0)	Accuracy: 82.16%
2	Validation losses: 0.5685, 3.1642, 3.7327	Best Loss: 3.7327 (0)	Accuracy: 80.12%
3	Validation losses: 0.5960, 1.7853, 2.3814	Best Loss: 2.3814 (0)	Accuracy: 79.08%
4	Validation losses: 0.5623, 1.2812, 1.8435	Best Loss: 1.8435 (0)	Accuracy: 80.20%
5	Validation losses: 0.6169, 1.0306, 1.6474	Best Loss: 1.6474 (0)	Accuracy: 77.94%
6	Validation losses: 0.4852, 0.8436, 1.3288	Best Loss: 1.3288 (0)	Accuracy: 83.04%
7	Validation losses: 0.7463, 0.7370, 1.4833	Best Loss: 1.3288 (1)	Accuracy: 74.30%
8	Validation losses: 0.5293, 0.6562, 1.1855	Best Loss: 1.1855 (0)	Accuracy: 81.48%
9	Validation losses: 0.5185, 0.6060, 1.1245	Best Loss: 1.1245 (0)	Accuracy: 81.42%
10	Validation losses: 0.5390, 0.5488, 1.0877	Best Loss: 1.0877 (0)	Accuracy: 81.00%
11	Validation losses: 0.50

21	Validation losses: nan, nan, nan	Best Loss: 0.9617 (7)	Accuracy: 0.00%
22	Validation losses: nan, nan, nan	Best Loss: 0.9617 (8)	Accuracy: 0.00%
23	Validation losses: nan, nan, nan	Best Loss: 0.9617 (9)	Accuracy: 0.00%
24	Validation losses: nan, nan, nan	Best Loss: 0.9617 (10)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 0.9617 (11)	Accuracy: 0.00%
26	Validation losses: nan, nan, nan	Best Loss: 0.9617 (12)	Accuracy: 0.00%
27	Validation losses: nan, nan, nan	Best Loss: 0.9617 (13)	Accuracy: 0.00%
28	Validation losses: nan, nan, nan	Best Loss: 0.9617 (14)	Accuracy: 0.00%
29	Validation losses: nan, nan, nan	Best Loss: 0.9617 (15)	Accuracy: 0.00%
30	Validation losses: nan, nan, nan	Best Loss: 0.9617 (16)	Accuracy: 0.00%
31	Validation losses: nan, nan, nan	Best Loss: 0.9617 (17)	Accuracy: 0.00%
32	Validation losses: nan, nan, nan	Best Loss: 0.9617 (18)	Accuracy: 0.00%
33	Validation losses: nan, nan, nan	Best Loss: 0.9617 (19)	Accuracy: 0.00%
34	Validation losses: nan, n

45	Validation losses: 0.4166, 0.3078, 0.7244	Best Loss: 0.7035 (5)	Accuracy: 85.06%
46	Validation losses: 0.4057, 0.3051, 0.7109	Best Loss: 0.7035 (6)	Accuracy: 85.40%
47	Validation losses: 0.4127, 0.3050, 0.7177	Best Loss: 0.7035 (7)	Accuracy: 85.36%
48	Validation losses: 0.4065, 0.3049, 0.7114	Best Loss: 0.7035 (8)	Accuracy: 85.76%
49	Validation losses: 0.4271, 0.3051, 0.7323	Best Loss: 0.7035 (9)	Accuracy: 85.52%
50	Validation losses: 0.4087, 0.3044, 0.7131	Best Loss: 0.7035 (10)	Accuracy: 85.72%
51	Validation losses: 0.4186, 0.3081, 0.7267	Best Loss: 0.7035 (11)	Accuracy: 85.64%
52	Validation losses: 0.5323, 0.3056, 0.8379	Best Loss: 0.7035 (12)	Accuracy: 79.30%
53	Validation losses: 0.4326, 0.3070, 0.7396	Best Loss: 0.7035 (13)	Accuracy: 84.30%
54	Validation losses: 0.4349, 0.3061, 0.7409	Best Loss: 0.7035 (14)	Accuracy: 84.10%
55	Validation losses: 0.4293, 0.3081, 0.7374	Best Loss: 0.7035 (15)	Accuracy: 84.94%
56	Validation losses: 0.4174, 0.3042, 0.7216	Best Loss: 0.7035 (16)	Ac

2	Validation losses: 0.4849, 4.9527, 5.4376	Best Loss: 5.4376 (0)	Accuracy: 83.38%
3	Validation losses: 0.5818, 2.2371, 2.8189	Best Loss: 2.8189 (0)	Accuracy: 78.32%
4	Validation losses: 0.4919, 1.3572, 1.8491	Best Loss: 1.8491 (0)	Accuracy: 82.02%
5	Validation losses: 0.5461, 0.9676, 1.5137	Best Loss: 1.5137 (0)	Accuracy: 80.38%
6	Validation losses: 0.5008, 0.7554, 1.2562	Best Loss: 1.2562 (0)	Accuracy: 82.94%
7	Validation losses: 0.5526, 0.6313, 1.1839	Best Loss: 1.1839 (0)	Accuracy: 81.60%
8	Validation losses: 0.5685, 0.5749, 1.1435	Best Loss: 1.1435 (0)	Accuracy: 79.60%
9	Validation losses: 0.4462, 0.4872, 0.9334	Best Loss: 0.9334 (0)	Accuracy: 84.16%
10	Validation losses: 0.4684, 0.4458, 0.9142	Best Loss: 0.9142 (0)	Accuracy: 82.98%
11	Validation losses: 0.4757, 0.4109, 0.8867	Best Loss: 0.8867 (0)	Accuracy: 83.94%
12	Validation losses: 0.4630, 0.3925, 0.8555	Best Loss: 0.8555 (0)	Accuracy: 83.94%
13	Validation losses: 0.4239, 0.3641, 0.7880	Best Loss: 0.7880 (0)	Accuracy: 84.86%


57	Validation losses: 0.4001, 0.2538, 0.6539	Best Loss: 0.6010 (16)	Accuracy: 85.48%
58	Validation losses: 0.4061, 0.2516, 0.6577	Best Loss: 0.6010 (17)	Accuracy: 85.68%
59	Validation losses: 0.4063, 0.2557, 0.6619	Best Loss: 0.6010 (18)	Accuracy: 85.98%
60	Validation losses: 0.3872, 0.2518, 0.6390	Best Loss: 0.6010 (19)	Accuracy: 86.52%
61	Validation losses: 0.3999, 0.2513, 0.6512	Best Loss: 0.6010 (20)	Accuracy: 85.70%
Early stopping!
Total running time:	 4907
INFO:tensorflow:Restoring parameters from ./models/model_20240511091914.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    14.2700004578%
RANDOM SEED:   1 	LAMBDA_EXP:   -4.6
Now: 20240511104102
0	Validation losses: 0.4251, 29.0327, 29.4579	Best Loss: 29.4579 (0)	Accuracy: 84.32%
1	Validation losses: 0.3802, 16.2555, 16.6357	Best Loss: 16.6357 (0)	Accuracy: 86.98%
2	Validation losses: 0.4078, 6.7339, 7.1417	Best Loss: 7.1417 (0)	A

...

36	Validation losses: nan, nan, nan	Best Loss: 0.6704 (19)	Accuracy: 0.00%
37	Validation losses: nan, nan, nan	Best Loss: 0.6704 (20)	Accuracy: 0.00%
Early stopping!
Total running time:	 3062
INFO:tensorflow:Restoring parameters from ./models/model_20240512230432.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    12.1599984169%
RANDOM SEED:   3 	LAMBDA_EXP:   -6.4
Now: 20240512235535
0	Validation losses: 0.4238, 0.5815, 1.0052	Best Loss: 1.0052 (0)	Accuracy: 85.50%
1	Validation losses: 0.3598, 0.5779, 0.9376	Best Loss: 0.9376 (0)	Accuracy: 87.38%
2	Validation losses: 0.3515, 0.5722, 0.9238	Best Loss: 0.9238 (0)	Accuracy: 87.44%
3	Validation losses: 0.3718, 0.5647, 0.9365	Best Loss: 0.9238 (1)	Accuracy: 86.30%
4	Validation losses: 0.3291, 0.5554, 0.8845	Best Loss: 0.8845 (0)	Accuracy: 87.48%
5	Validation losses: 0.3282, 0.5445, 0.8727	Best Loss: 0.8727 (0)	Accuracy: 87.68%
6	Validation los

38	Validation losses: 0.6122, 0.1840, 0.7961	Best Loss: 0.5565 (12)	Accuracy: 90.48%
39	Validation losses: 0.6415, 0.1832, 0.8247	Best Loss: 0.5565 (13)	Accuracy: 90.56%
40	Validation losses: 0.6654, 0.1828, 0.8483	Best Loss: 0.5565 (14)	Accuracy: 90.72%
41	Validation losses: 0.6803, 0.1823, 0.8625	Best Loss: 0.5565 (15)	Accuracy: 90.54%
42	Validation losses: 0.7068, 0.1815, 0.8883	Best Loss: 0.5565 (16)	Accuracy: 90.64%
43	Validation losses: 0.7079, 0.1805, 0.8884	Best Loss: 0.5565 (17)	Accuracy: 90.62%
44	Validation losses: 0.7297, 0.1794, 0.9092	Best Loss: 0.5565 (18)	Accuracy: 90.34%
45	Validation losses: 0.6696, 0.1783, 0.8479	Best Loss: 0.5565 (19)	Accuracy: 89.70%
46	Validation losses: 0.6356, 0.1771, 0.8127	Best Loss: 0.5565 (20)	Accuracy: 90.14%
Early stopping!
Total running time:	 3890
INFO:tensorflow:Restoring parameters from ./models/model_20240513010423.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remainin

33	Validation losses: 0.4263, 0.1938, 0.6201	Best Loss: 0.5506 (6)	Accuracy: 90.58%
34	Validation losses: 0.5022, 0.1913, 0.6935	Best Loss: 0.5506 (7)	Accuracy: 89.86%
35	Validation losses: 0.4986, 0.1892, 0.6878	Best Loss: 0.5506 (8)	Accuracy: 90.46%
36	Validation losses: 0.5337, 0.1873, 0.7210	Best Loss: 0.5506 (9)	Accuracy: 90.52%
37	Validation losses: 0.5664, 0.1858, 0.7521	Best Loss: 0.5506 (10)	Accuracy: 90.64%
38	Validation losses: 0.6246, 0.1846, 0.8092	Best Loss: 0.5506 (11)	Accuracy: 90.40%
39	Validation losses: 0.6472, 0.1838, 0.8310	Best Loss: 0.5506 (12)	Accuracy: 90.64%
40	Validation losses: 0.6658, 0.1835, 0.8493	Best Loss: 0.5506 (13)	Accuracy: 90.70%
41	Validation losses: 0.6948, 0.1829, 0.8777	Best Loss: 0.5506 (14)	Accuracy: 90.90%
42	Validation losses: 0.7042, 0.1822, 0.8864	Best Loss: 0.5506 (15)	Accuracy: 90.76%
43	Validation losses: 0.7295, 0.1812, 0.9107	Best Loss: 0.5506 (16)	Accuracy: 90.62%
44	Validation losses: 0.6926, 0.1801, 0.8727	Best Loss: 0.5506 (17)	A

2	Validation losses: 0.3652, 0.2311, 0.5963	Best Loss: 0.5926 (1)	Accuracy: 86.86%
3	Validation losses: 0.3783, 0.2304, 0.6087	Best Loss: 0.5926 (2)	Accuracy: 86.18%
4	Validation losses: 0.3207, 0.2294, 0.5501	Best Loss: 0.5501 (0)	Accuracy: 87.88%
5	Validation losses: 0.3414, 0.2283, 0.5697	Best Loss: 0.5501 (1)	Accuracy: 87.62%
6	Validation losses: 0.3434, 0.2270, 0.5704	Best Loss: 0.5501 (2)	Accuracy: 87.64%
7	Validation losses: 0.3583, 0.2256, 0.5838	Best Loss: 0.5501 (3)	Accuracy: 87.02%
8	Validation losses: 0.2935, 0.2240, 0.5175	Best Loss: 0.5175 (0)	Accuracy: 89.66%
9	Validation losses: 0.3515, 0.2223, 0.5738	Best Loss: 0.5175 (1)	Accuracy: 87.82%
10	Validation losses: 0.3560, 0.2204, 0.5764	Best Loss: 0.5175 (2)	Accuracy: 87.36%
11	Validation losses: 0.3811, 0.2185, 0.5996	Best Loss: 0.5175 (3)	Accuracy: 84.78%
12	Validation losses: 0.3308, 0.2164, 0.5472	Best Loss: 0.5175 (4)	Accuracy: 88.64%
13	Validation losses: 0.2981, 0.2142, 0.5123	Best Loss: 0.5123 (0)	Accuracy: 88.98%


26	Validation losses: 0.3598, 0.1367, 0.4965	Best Loss: 0.4470 (18)	Accuracy: 89.76%
27	Validation losses: 0.3544, 0.1357, 0.4902	Best Loss: 0.4470 (19)	Accuracy: 90.16%
28	Validation losses: 0.3748, 0.1349, 0.5097	Best Loss: 0.4470 (20)	Accuracy: 90.40%
Early stopping!
Total running time:	 2440
INFO:tensorflow:Restoring parameters from ./models/model_20240513073121.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.5299999714%
RANDOM SEED:   3 	LAMBDA_EXP:   -7.0
Now: 20240513081202
0	Validation losses: 0.4287, 0.1465, 0.5751	Best Loss: 0.5751 (0)	Accuracy: 85.20%
1	Validation losses: 0.3587, 0.1464, 0.5051	Best Loss: 0.5051 (0)	Accuracy: 86.92%
2	Validation losses: 0.3523, 0.1463, 0.4986	Best Loss: 0.4986 (0)	Accuracy: 87.42%
3	Validation losses: 0.3782, 0.1462, 0.5244	Best Loss: 0.4986 (1)	Accuracy: 86.26%
4	Validation losses: 0.3260, 0.1460, 0.4720	Best Loss: 0.4720 (0)	Accuracy: 8